In [3]:
# Install required libraries
!pip install sentence-transformers spacy nltk
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.3 MB/s eta 0:00:00:00:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import json
from tqdm import tqdm
import logging
import time
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, roc_auc_score, roc_curve
import spacy
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import Counter

2025-04-15 04:35:57.790981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744691757.970822      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744691758.025310      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
 #Download VADER lexicon
import torch.nn.functional as F
nltk.download('vader_lexicon')

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# Load spaCy and VADER
logger.info("Loading spaCy and VADER...")
start_time = time.time()
nlp = spacy.load('en_core_web_sm', disable=['ner', 'lemmatizer'])
sid = SentimentIntensityAnalyzer()
logger.info(f"spaCy and VADER loaded in {time.time() - start_time:.2f} seconds")

# Load Sentence-BERT model
logger.info("Loading Sentence-BERT model...")
start_time = time.time()
sentence_bert = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)
sentence_bert.eval()
logger.info(f"Sentence-BERT loaded in {time.time() - start_time:.2f} seconds")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [39]:
# Set seed for reproducibility
import random

seed = 100

torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark=False

In [40]:

# Model Definition
class EnhancedDeceptionDetector(nn.Module):
    def __init__(self, num_countries, num_seasons, num_years, 
                 sentence_emb_dim=384, ling_feature_dim=6, country_emb_dim=16, 
                 season_emb_dim=8, year_emb_dim=8, hidden_dim=64, dropout=0.4):
        super(EnhancedDeceptionDetector, self).__init__()
        self.ling_feature_dim = ling_feature_dim
        self.country_embedding = nn.Embedding(num_countries + 1, country_emb_dim, padding_idx=num_countries)
        self.season_embedding = nn.Embedding(num_seasons + 1, season_emb_dim, padding_idx=num_seasons)
        self.year_embedding = nn.Embedding(num_years + 1, year_emb_dim, padding_idx=num_years)
        
        input_dim = sentence_emb_dim + ling_feature_dim + (country_emb_dim * 2) + season_emb_dim + year_emb_dim + 2
        self.ling_projection = nn.Linear(ling_feature_dim, ling_feature_dim) if ling_feature_dim > 0 else None
        self.input_projection = nn.Linear(input_dim, hidden_dim)
        
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.layer_norm = nn.LayerNorm(hidden_dim * 2)
        self.classifier = nn.Linear(hidden_dim * 2, 1)
        self.dropout = nn.Dropout(dropout)
        self.attention = nn.Linear(hidden_dim * 2, 1)

    def forward(self, messages_list, ling_features, senders, receivers, seasons, years, 
                game_scores, game_score_deltas, lengths):
        batch_size = len(messages_list)
        
        message_embs_list = []
        for batch_idx in range(batch_size):
            msgs = ['' if msg == '[EMPTY]' else msg for msg in messages_list[batch_idx]]
            with torch.no_grad():
                embs = sentence_bert.encode(
                    msgs, convert_to_tensor=True, device=device, show_progress_bar=False, batch_size=32
                )
            message_embs_list.append(embs)
        
        message_embs_padded = pad_sequence(message_embs_list, batch_first=True, padding_value=0)
        
        if self.ling_feature_dim > 0:
            ling_features_padded = pad_sequence(ling_features, batch_first=True, padding_value=0).to(device)
            ling_features_padded = self.ling_projection(ling_features_padded)
        else:
            ling_features_padded = torch.zeros(batch_size, message_embs_padded.size(1), 0, device=device)
        
        senders_emb = self.country_embedding(senders)
        receivers_emb = self.country_embedding(receivers)
        seasons_emb = self.season_embedding(seasons)
        years_emb = self.year_embedding(years)
        game_scores = game_scores.unsqueeze(-1)
        game_score_deltas = game_score_deltas.unsqueeze(-1)

        combined = torch.cat([
            message_embs_padded, ling_features_padded, senders_emb, receivers_emb,
            seasons_emb, years_emb, game_scores, game_score_deltas
        ], dim=2)
        
        combined = self.input_projection(combined)
        combined = self.dropout(combined)
        
        packed_input = pack_padded_sequence(combined, lengths, batch_first=True, enforce_sorted=False)
        lstm_out_packed, _ = self.lstm(packed_input)
        lstm_out, _ = pad_packed_sequence(lstm_out_packed, batch_first=True)
        
        lstm_out = self.layer_norm(lstm_out)
        attn_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        context = self.layer_norm(context)
        final_pred = self.classifier(context).unsqueeze(1)
        return final_pred.expand(-1, lstm_out.size(1), -1)

# Data Preprocessing
def preprocess_data(data_file, country_map=None, season_map=None, year_map=None, use_ling_features=True, ling_stats=None):
    logger.info(f"Preprocessing {data_file}")
    with open(data_file, 'r') as f:
        data = [json.loads(line) for line in f]
    
    if country_map is None:
        countries = set()
        for game in data:
            countries.update(game['speakers'])
            countries.update(game['receivers'])
        country_map = {c: i for i, c in enumerate(sorted(countries))}
        logger.info(f"Created country map with {len(country_map)} countries")
    
    if season_map is None:
        seasons = set()
        for game in data:
            seasons.update(game['seasons'])
        season_map = {s: i for i, s in enumerate(sorted(seasons))}
        logger.info(f"Created season map with {len(season_map)} seasons")
    
    if year_map is None:
        years = set()
        for game in data:
            years.update(game['years'])
        year_map = {y: i for i, y in enumerate(sorted(years))}
        logger.info(f"Created year map with {len(year_map)} years")
    
    conversations = []
    ling_features_all = [] if use_ling_features else None
    for game in tqdm(data, desc="Processing conversations"):
        messages = [msg if msg else '[EMPTY]' for msg in game['messages']]
        if len(messages) == 0:
            continue
        senders = [country_map[s] for s in game['speakers']]
        receivers = [country_map[r] for r in game['receivers']]
        seasons = [season_map[s] for s in game['seasons']]
        years = [year_map[y] for y in game['years']]
        game_scores = [float(gs) for gs in game['game_score']]
        game_score_deltas = [float(gsd) for gsd in game['game_score_delta']]
        sender_labels = [1 if l else 0 for l in game['sender_labels']]
        receiver_labels = [-1 if l == "NOANNOTATION" else (1 if l else 0) for l in game['receiver_labels']]
        
        ling_features = []
        for msg in messages:
            if msg == '[EMPTY]' or not use_ling_features:
                ling_features.append([0.0] * 6)
            else:
                doc = nlp(msg)
                words = [token.text.lower() for token in doc if not token.is_punct]
                word_count = min(len(words), 100)
                lexical_diversity = len(set(words)) / (len(words) + 1e-8) if len(words) > 0 else 0
                lexical_diversity = min(lexical_diversity, 1.0)
                self_refs = min(sum(1 for token in doc if token.text.lower() in ['i', 'me', 'my', 'mine', 'myself']), 10)
                group_refs = min(sum(1 for token in doc if token.text.lower() in ['we', 'us', 'our', 'ours', 'ourselves']), 10)
                neg_emotion = sid.polarity_scores(msg)['neg']
                modifiers = min(sum(1 for token in doc if token.pos_ in ['ADJ', 'ADV']), 10)
                ling_features.append([
                    np.log1p(word_count), lexical_diversity, self_refs, group_refs, neg_emotion, modifiers
                ])
            if data_file.endswith('train.jsonl'):
                ling_features_all.append(ling_features[-1])
        
        conversations.append({
            'messages': messages, 'ling_features': torch.tensor(ling_features, dtype=torch.float32),
            'speakers': torch.tensor(senders, dtype=torch.long), 'receivers': torch.tensor(receivers, dtype=torch.long),
            'seasons': torch.tensor(seasons, dtype=torch.long), 'years': torch.tensor(years, dtype=torch.long),
            'game_scores': torch.tensor(game_scores, dtype=torch.float32),
            'game_score_deltas': torch.tensor(game_score_deltas, dtype=torch.float32),
            'sender_labels': torch.tensor(sender_labels, dtype=torch.float32),
            'receiver_labels': torch.tensor(receiver_labels, dtype=torch.float32)
        })
    
    if use_ling_features and data_file.endswith('train.jsonl'):
        ling_features_all = np.array(ling_features_all)
        ling_stats = {'mean': np.mean(ling_features_all, axis=0), 'std': np.std(ling_features_all, axis=0) + 1e-8}
    elif use_ling_features and ling_stats is None:
        raise ValueError("ling_stats must be provided for val/test when use_ling_features=True")
    
    if use_ling_features:
        for conv in conversations:
            ling_features = conv['ling_features'].numpy()
            ling_features = (ling_features - ling_stats['mean']) / ling_stats['std']
            conv['ling_features'] = torch.tensor(ling_features, dtype=torch.float32)
    
    logger.info(f"Processed {len(conversations)} conversations from {data_file}")
    return conversations, country_map, season_map, year_map, ling_stats

# Oversample Lie Conversations
def oversample_lies(data, oversample_factor=2):
    lie_convs = [conv for conv in data if any(l == 1 for l in conv['sender_labels'])]
    oversampled = data + lie_convs * (oversample_factor - 1)
    logger.info(f"Oversampled: {len(data)} -> {len(oversampled)} conversations")
    return oversampled

# Compute Metrics
def compute_metrics(preds, labels, mask=None):
    if mask is not None:
        preds = preds[mask.bool()]
        labels = labels[mask.bool()]
    preds = (preds > 0).float()
    labels = labels.float()
    
    precision, recall, fscore, _ = precision_recall_fscore_support(labels.cpu(), preds.cpu(), average=None, labels=[0, 1], zero_division=0)
    micro = precision_recall_fscore_support(labels.cpu(), preds.cpu(), average='micro', zero_division=0)[:3]
    macro = precision_recall_fscore_support(labels.cpu(), preds.cpu(), average='macro', zero_division=0)[:3]
    conf_matrix = confusion_matrix(labels.cpu(), preds.cpu(), labels=[0, 1])
    
    return {
        'True_precision': precision[1], 'False_precision': precision[0],
        'True_recall': recall[1], 'False_recall': recall[0],
        'True_fscore': fscore[1], 'False_fscore': fscore[0],
        'micro_precision': micro[0], 'micro_recall': micro[1], 'micro_fscore': micro[2],
        'macro_precision': macro[0], 'macro_recall': macro[1], 'macro_fscore': macro[2],
        'confusion_matrix': conf_matrix
    }

# Training and Evaluation
def train_and_evaluate(model, train_data, val_data, test_data, epochs=15, batch_size=4, lr=0.0001, patience=3,
                      checkpoint_dir='checkpoints', start_epoch=0, checkpoint=None):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.01)
    os.makedirs(checkpoint_dir, exist_ok=True)
    warmup_steps = int(0.1 * epochs * (len(train_data) / batch_size))
    def lr_lambda(step):
        if step < warmup_steps:
            return float(step) / warmup_steps
        return 1.0
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    if checkpoint is not None:
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch']
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([checkpoint['pos_weight']], device=device), reduction='none')
        print(f"Resuming training from epoch {start_epoch}")
    else:
        all_labels = [l.item() for conv in train_data for l in conv['sender_labels'] if l != -1]
        num_lies = sum(l == 1 for l in all_labels)
        num_truths = len(all_labels) - num_lies
        pos_weight_val = (num_truths / num_lies) * 1.5 if num_lies > 0 else 1.0
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight_val], device=device), reduction='none')
        model.classifier.bias.data.fill_(1.0)

    # Track best metrics for best_checkpoint.pt
    best_macro_f1 = -float('inf')
    best_false_f1 = -float('inf')
    best_val_loss = float('inf') if checkpoint is None else checkpoint['val_loss']
    best_metrics = None if checkpoint is None else checkpoint['metrics']
    best_model_state = None if checkpoint is None else model.state_dict().copy()
    epochs_no_improve = 0
    best_pos_weight = pos_weight_val if checkpoint is None else checkpoint['pos_weight']

    def evaluate_batch(data, batch_size=4):
        model.eval()
        total_loss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for i in range(0, len(data), batch_size):
                batch = data[i:i+batch_size]
                messages = [c['messages'] for c in batch]
                ling_features = [c['ling_features'] for c in batch]
                senders = torch.nn.utils.rnn.pad_sequence([c['speakers'] for c in batch], batch_first=True, padding_value=model.country_embedding.padding_idx).to(device)
                receivers = torch.nn.utils.rnn.pad_sequence([c['receivers'] for c in batch], batch_first=True, padding_value=model.country_embedding.padding_idx).to(device)
                seasons = torch.nn.utils.rnn.pad_sequence([c['seasons'] for c in batch], batch_first=True, padding_value=model.season_embedding.padding_idx).to(device)
                years = torch.nn.utils.rnn.pad_sequence([c['years'] for c in batch], batch_first=True, padding_value=model.year_embedding.padding_idx).to(device)
                game_scores = torch.nn.utils.rnn.pad_sequence([c['game_scores'] for c in batch], batch_first=True, padding_value=0).to(device)
                game_score_deltas = torch.nn.utils.rnn.pad_sequence([c['game_score_deltas'] for c in batch], batch_first=True, padding_value=0).to(device)
                sender_labels = torch.nn.utils.rnn.pad_sequence([c['sender_labels'] for c in batch], batch_first=True, padding_value=-1).to(device)
                lengths = [len(c['messages']) for c in batch]

                final_pred = model(messages, ling_features, senders, receivers, seasons, years,
                                  game_scores, game_score_deltas, lengths)
                loss = criterion(final_pred.squeeze(-1), sender_labels)
                mask = (sender_labels != -1)
                loss = (loss * mask).sum() / (mask.sum() + 1e-8)
                total_loss += loss.item()

                all_preds.append(final_pred.squeeze(-1)[mask])
                all_labels.append(sender_labels[mask])

        avg_loss = total_loss / (len(data) / batch_size)
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        metrics = compute_metrics(all_preds, all_labels)
        return avg_loss, metrics

    for epoch in range(start_epoch, epochs):
        model.train()
        total_loss = 0
        for i in range(0, len(train_data), batch_size):
            optimizer.zero_grad()
            batch = train_data[i:i+batch_size]
            messages = [c['messages'] for c in batch]
            ling_features = [c['ling_features'] for c in batch]
            senders = torch.nn.utils.rnn.pad_sequence([c['speakers'] for c in batch], batch_first=True, padding_value=model.country_embedding.padding_idx).to(device)
            receivers = torch.nn.utils.rnn.pad_sequence([c['receivers'] for c in batch], batch_first=True, padding_value=model.country_embedding.padding_idx).to(device)
            seasons = torch.nn.utils.rnn.pad_sequence([c['seasons'] for c in batch], batch_first=True, padding_value=model.season_embedding.padding_idx).to(device)
            years = torch.nn.utils.rnn.pad_sequence([c['years'] for c in batch], batch_first=True, padding_value=model.year_embedding.padding_idx).to(device)
            game_scores = torch.nn.utils.rnn.pad_sequence([c['game_scores'] for c in batch], batch_first=True, padding_value=0).to(device)
            game_score_deltas = torch.nn.utils.rnn.pad_sequence([c['game_score_deltas'] for c in batch], batch_first=True, padding_value=0).to(device)
            sender_labels = torch.nn.utils.rnn.pad_sequence([c['sender_labels'] for c in batch], batch_first=True, padding_value=-1).to(device)
            lengths = [len(c['messages']) for c in batch]

            final_pred = model(messages, ling_features, senders, receivers, seasons, years,
                              game_scores, game_score_deltas, lengths)
            loss = criterion(final_pred.squeeze(-1), sender_labels)
            mask = (sender_labels != -1)
            loss = (loss * mask).sum() / (mask.sum() + 1e-8)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_loss / (len(train_data) / batch_size)

        # Validate
        val_loss, val_metrics = evaluate_batch(val_data)
        print(f"\n=== Epoch {epoch+1}/{epochs} Metrics ===")
        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Validation Loss: {val_loss:.4f}")
        print("\nPer-Class Metrics:")
        print(f"True Precision: {val_metrics['True_precision']:.4f}, False Precision: {val_metrics['False_precision']:.4f}")
        print(f"True Recall: {val_metrics['True_recall']:.4f}, False Recall: {val_metrics['False_recall']:.4f}")
        print(f"True F1: {val_metrics['True_fscore']:.4f}, False F1: {val_metrics['False_fscore']:.4f}")
        print("\nAggregated Metrics:")
        print(f"Micro F1: {val_metrics['micro_fscore']:.4f}, Macro F1: {val_metrics['macro_fscore']:.4f}")
        print(f"Confusion Matrix:\n{val_metrics['confusion_matrix']}")

        # Save epoch checkpoint if criteria met
       # Save epoch checkpoint if criteria met
        if val_metrics['macro_fscore'] > 0.55 or val_metrics['False_fscore'] > 0.18:
            checkpoint_state = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_loss': val_loss,
                'metrics': val_metrics,
                'pos_weight': pos_weight_val
            }
            checkpoint_filename = f'checkpoint_epoch_{epoch+1}_macroF1_{val_metrics["macro_fscore"]:.4f}_falseF1_{val_metrics["False_fscore"]:.4f}.pt'
            torch.save(checkpoint_state, os.path.join(checkpoint_dir, checkpoint_filename))
            print(f"Saved checkpoint {checkpoint_filename} (False F1: {val_metrics['False_fscore']:.4f}, Macro F1: {val_metrics['macro_fscore']:.4f})")

# Update best_checkpoint.pt based on Macro F1 (or False F1 as tiebreaker)
    
        if val_metrics['macro_fscore'] > best_macro_f1 or \
           (val_metrics['macro_fscore'] == best_macro_f1 and val_metrics['False_fscore'] > best_false_f1):
            best_macro_f1 = val_metrics['macro_fscore']
            best_false_f1 = val_metrics['False_fscore']
            best_val_loss = val_loss
            best_metrics = val_metrics
            best_model_state = model.state_dict().copy()
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': best_model_state,
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_loss': best_val_loss,
                'metrics': best_metrics,
                'pos_weight': pos_weight_val
            }, os.path.join(checkpoint_dir, 'best_checkpoint.pt'))
            print(f"Updated best_checkpoint.pt with Macro F1: {best_macro_f1:.4f}, False F1: {best_false_f1:.4f}")
    # Load best model for final evaluation
    model.load_state_dict(best_model_state)
    test_loss, test_metrics = evaluate_batch(test_data)
    print(f"\n=== Final Test Metrics ===")
    print(f"Test Loss: {test_loss:.4f}")
    print("\nPer-Class Metrics:")
    print(f"True Precision: {test_metrics['True_precision']:.4f}, False Precision: {test_metrics['False_precision']:.4f}")
    print(f"True Recall: {test_metrics['True_recall']:.4f}, False Recall: {test_metrics['False_recall']:.4f}")
    print(f"True F1: {test_metrics['True_fscore']:.4f}, False F1: {test_metrics['False_fscore']:.4f}")
    print("\nAggregated Metrics:")
    print(f"Micro F1: {test_metrics['micro_fscore']:.4f}, Macro F1: {test_metrics['macro_fscore']:.4f}")
    print(f"Confusion Matrix:\n{test_metrics['confusion_matrix']}")
    return {'test_loss': test_loss, 'test_metrics': test_metrics, 'training_loss': avg_train_loss,
            'validation_loss': val_loss, 'best_validation_loss': best_val_loss,
            'best_validation_metrics': best_metrics}

# Grid Search for Hyperparameters
def grid_search_hyperparams(train_data, val_data, test_data, num_countries, num_seasons, num_years, 
                           use_ling_features=True, epochs=15, patience=3):
    param_grid = {'lr': [0.00005, 0.0001, 0.0002], 'batch_size': [4, 8]}
    best_macro_f1 = 0
    best_params = None
    best_results = None
    
    for lr, batch_size in itertools.product(param_grid['lr'], param_grid['batch_size']):
        logger.info(f"Testing lr={lr}, batch_size={batch_size}")
        model = EnhancedDeceptionDetector(
            num_countries=num_countries, num_seasons=num_seasons, num_years=num_years,
            sentence_emb_dim=384, ling_feature_dim=6 if use_ling_features else 0,
            country_emb_dim=16, season_emb_dim=8, year_emb_dim=8, hidden_dim=64, dropout=0.4
        ).to(device)
        
        results = train_and_evaluate(model, train_data, val_data, test_data,
                                     epochs=epochs, batch_size=batch_size, lr=lr, patience=patience)
        
        macro_f1 = results['best_validation_metrics']['macro_fscore']
        logger.info(f"lr={lr}, batch_size={batch_size}, Validation Macro F1: {macro_f1:.4f}")
        
        if macro_f1 > best_macro_f1:
            best_macro_f1 = macro_f1
            best_params = {'lr': lr, 'batch_size': batch_size}
            best_results = results
    
    return best_params, best_results

# Oversample Lie Conversations
def oversample_lies(data, oversample_factor=2):
    lie_convs = [conv for conv in data if any(l == 1 for l in conv['sender_labels'])]
    oversampled = data + lie_convs * (oversample_factor - 1)
    logger.info(f"Oversampled: {len(data)} -> {len(oversampled)} conversations")
    return oversampled

# Main Execution
if __name__ == "__main__":
    train_file = "/kaggle/input/dataset-deception/train.jsonl"
    val_file = "/kaggle/input/dataset-deception/validation.jsonl"
    test_file = "/kaggle/input/dataset-deception/test.jsonl"
    
    use_ling_features = True
    train_data, country_map, season_map, year_map, ling_stats = preprocess_data(train_file, use_ling_features=use_ling_features)
    val_data, _, _, _, _ = preprocess_data(val_file, country_map, season_map, year_map, use_ling_features=use_ling_features, ling_stats=ling_stats)
    test_data, _, _, _, _ = preprocess_data(test_file, country_map, season_map, year_map, use_ling_features=use_ling_features, ling_stats=ling_stats)
    
    train_data = oversample_lies(train_data, oversample_factor=2)
    
    num_countries = len(country_map)
    num_seasons = len(season_map)
    num_years = len(year_map)
    best_params, results = grid_search_hyperparams(
        train_data, val_data, test_data, num_countries, num_seasons, num_years,
        use_ling_features=use_ling_features, epochs=15, patience=3
    )
    
    logger.info(f"Best parameters: lr={best_params['lr']}, batch_size={best_params['batch_size']}")
    
    print("\n=== Per-Class Metrics (for 'True' and 'False' classes) ===")
    print("Precision")
    print(f"True_precision: {results['test_metrics']['True_precision']:.4f}")
    print(f"False_precision: {results['test_metrics']['False_precision']:.4f}")
    print("Recall")
    print(f"True_recall: {results['test_metrics']['True_recall']:.4f}")
    print(f"False_recall: {results['test_metrics']['False_recall']:.4f}")
    print("F1-Score")
    print(f"True_fscore: {results['test_metrics']['True_fscore']:.4f}")
    print(f"False_fscore: {results['test_metrics']['False_fscore']:.4f}")
    
    print("\n📊 Micro-Averaged Metrics")
    print(f"micro_precision: {results['test_metrics']['micro_precision']:.4f}")
    print(f"micro_recall: {results['test_metrics']['micro_recall']:.4f}")
    print(f"micro_fscore: {results['test_metrics']['micro_fscore']:.4f}")
    
    print("\n📈 Macro-Averaged Metrics")
    print(f"macro_precision: {results['test_metrics']['macro_precision']:.4f}")
    print(f"macro_recall: {results['test_metrics']['macro_recall']:.4f}")
    print(f"macro_fscore: {results['test_metrics']['macro_fscore']:.4f}")
    
    print("\n🧠 Loss")
    print(f"training_loss: {results['training_loss']:.4f}")
    print(f"validation_loss: {results['validation_loss']:.4f}")
    print(f"best_validation_loss: {results['best_validation_loss']:.4f}")
    print(f"test_loss: {results['test_loss']:.4f}")
    
    print("\n🧪 Best Validation Metrics")
    for metric in ['True_precision', 'False_precision', 'True_recall', 'False_recall',
                  'True_fscore', 'False_fscore', 'micro_precision', 'micro_recall',
                  'micro_fscore', 'macro_precision', 'macro_recall', 'macro_fscore']:
        print(f"best_validation_{metric}: {results['best_validation_metrics'][metric]:.4f}")
    
    print("\n🔍 Test Confusion Matrix")
    print(results['test_metrics']['confusion_matrix'])

Processing conversations: 100%|██████████| 42/42 [00:10<00:00,  3.93it/s]



=== Epoch 1/15 Metrics ===
Train Loss: 0.0902
Validation Loss: 0.0804

Per-Class Metrics:
True Precision: 0.9529, False Precision: 0.0103
True Recall: 0.7890, False Recall: 0.0536
True F1: 0.8632, False F1: 0.0173

Aggregated Metrics:
Micro F1: 0.7599, Macro F1: 0.4403
Confusion Matrix:
[[   3   53]
 [ 287 1073]]
Updated best_checkpoint.pt with Macro F1: 0.4403, False F1: 0.0173

=== Epoch 2/15 Metrics ===
Train Loss: 0.0858
Validation Loss: 0.0777

Per-Class Metrics:
True Precision: 0.9622, False Precision: 0.2000
True Recall: 0.9912, False Recall: 0.0536
True F1: 0.9765, False F1: 0.0845

Aggregated Metrics:
Micro F1: 0.9541, Macro F1: 0.5305
Confusion Matrix:
[[   3   53]
 [  12 1348]]
Updated best_checkpoint.pt with Macro F1: 0.5305, False F1: 0.0845

=== Epoch 3/15 Metrics ===
Train Loss: 0.0840
Validation Loss: 0.0758

Per-Class Metrics:
True Precision: 0.9657, False Precision: 0.2000
True Recall: 0.9735, False Recall: 0.1607
True F1: 0.9696, False F1: 0.1782

Aggregated Metrics

In [41]:
import os
print(os.listdir('checkpoints'))

['checkpoint_epoch_3_macroF1_0.5734_falseF1_0.1884.pt', 'checkpoint_epoch_13.pt', 'checkpoint_epoch_9_macroF1_0.5734_falseF1_0.1884.pt', 'checkpoint_epoch_12.pt', 'checkpoint_epoch_3_macroF1_0.5739_falseF1_0.1782.pt', 'checkpoint_epoch_7_macroF1_0.5734_falseF1_0.1884.pt', 'checkpoint_epoch_2_macroF1_0.5817_falseF1_0.2016.pt', 'checkpoint_epoch_14_macroF1_0.5504_falseF1_0.1573.pt', 'checkpoint_epoch_6.pt', 'model_lr5e-05_bs4.pth', 'checkpoint_epoch_6_macroF1_0.5817_falseF1_0.2016.pt', 'checkpoint_epoch_4_macroF1_0.5739_falseF1_0.1782.pt', 'checkpoint_epoch_5_macroF1_0.5734_falseF1_0.1884.pt', 'checkpoint_epoch_5_macroF1_0.5739_falseF1_0.1782.pt', 'checkpoint_epoch_5_macroF1_0.5558_falseF1_0.1527.pt', 'checkpoint_epoch_13_macroF1_0.5564_falseF1_0.1657.pt', 'checkpoint_epoch_4_macroF1_0.5558_falseF1_0.1527.pt', 'checkpoint_epoch_2.pt', 'model_lr0.0001_bs4.pth', 'model_lr0.0002_bs8.pth', 'checkpoint_epoch_8.pt', 'model_lr5e-05_bs8.pth', 'model_lr0.0001_bs8.pth', 'checkpoint_epoch_15.pt', '

In [42]:
import os
import zipfile

# Define the directory containing checkpoints
checkpoint_dir = 'checkpoints'

# Define the output zip file name
zip_file_name = 'checkpoints_backup.zip'

# Create a zip file
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file_name in os.listdir(checkpoint_dir):
        file_path = os.path.join(checkpoint_dir, file_name)
        if os.path.isfile(file_path):  # Ensure it's a file
            zipf.write(file_path, os.path.relpath(file_path, os.path.dirname(checkpoint_dir)))

print(f"Created {zip_file_name} with all checkpoint files.")

Created checkpoints_backup.zip with all checkpoint files.


In [43]:
import os
print(os.listdir('/kaggle/working/'))

['checkpoints', 'checkpoints_backup.zip', '.ipynb_checkpoints', '.virtual_documents']


In [59]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import glob

# Get a list of all checkpoint files in the 'checkpoints' directory
checkpoint_files = glob.glob('checkpoints/*.pt')

def evaluate_model(model, checkpoint_path, data, batch_size=4):
    """
    Load a checkpoint and evaluate the model on the provided data, ensuring metrics match those saved.
    
    Args:
        model (EnhancedDeceptionDetector): The model instance to evaluate.
        checkpoint_path (str): Path to the saved checkpoint file.
        data (list): Preprocessed data (e.g., val_data) to evaluate on.
        batch_size (int): Batch size for evaluation.
    """
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()

    # Define the loss function with the checkpoint's pos_weight
    pos_weight = checkpoint['pos_weight']
    criterion = nn.BCEWithLogitsLoss(
        pos_weight=torch.tensor([pos_weight], device=device), reduction='none'
    )

    # Evaluation loop
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            messages = [c['messages'] for c in batch]
            ling_features = [c['ling_features'] for c in batch]
            senders = pad_sequence(
                [c['speakers'] for c in batch], batch_first=True, 
                padding_value=model.country_embedding.padding_idx
            ).to(device)
            receivers = pad_sequence(
                [c['receivers'] for c in batch], batch_first=True, 
                padding_value=model.country_embedding.padding_idx
            ).to(device)
            seasons = pad_sequence(
                [c['seasons'] for c in batch], batch_first=True, 
                padding_value=model.season_embedding.padding_idx
            ).to(device)
            years = pad_sequence(
                [c['years'] for c in batch], batch_first=True, 
                padding_value=model.year_embedding.padding_idx
            ).to(device)
            game_scores = pad_sequence(
                [c['game_scores'] for c in batch], batch_first=True, padding_value=0
            ).to(device)
            game_score_deltas = pad_sequence(
                [c['game_score_deltas'] for c in batch], batch_first=True, padding_value=0
            ).to(device)
            sender_labels = pad_sequence(
                [c['sender_labels'] for c in batch], batch_first=True, padding_value=-1
            ).to(device)
            lengths = [len(c['messages']) for c in batch]

            # Forward pass
            final_pred = model(
                messages, ling_features, senders, receivers, seasons, years,
                game_scores, game_score_deltas, lengths
            )
            loss = criterion(final_pred.squeeze(-1), sender_labels)
            mask = (sender_labels != -1)
            loss = (loss * mask).sum() / (mask.sum() + 1e-8)
            total_loss += loss.item()

            all_preds.append(final_pred.squeeze(-1)[mask])
            all_labels.append(sender_labels[mask])

    # Compute average loss and metrics
    avg_loss = total_loss / (len(data) / batch_size)
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    metrics = compute_metrics(all_preds, all_labels)

    # Print results
    print(f"Loaded checkpoint from epoch {checkpoint['epoch']}")
    print(f"Stored validation loss: {checkpoint['val_loss']:.4f}")
    print(f"Computed validation loss: {avg_loss:.4f}")
    print(f"Stored metrics: {checkpoint['metrics']}")
    print(f"Computed metrics: {metrics}")

    # Optional verification (can be removed if strict equality isn't required)
    # assert np.isclose(avg_loss, checkpoint['val_loss'], rtol=1e-5), "Validation loss mismatch"
    for key in metrics:
        if key != 'confusion_matrix':  # Confusion matrix is a numpy array
            assert np.isclose(metrics[key], checkpoint['metrics'][key], rtol=1e-5), f"Mismatch in {key}"
        else:
            assert np.array_equal(metrics[key], checkpoint['metrics'][key]), "Confusion matrix mismatch"

for checkpoint_file in checkpoint_files:
    print(f"\nEvaluating checkpoint: {checkpoint_file}")
    evaluate_model(model, checkpoint_file, val_data, batch_size=4)


Evaluating checkpoint: checkpoints/checkpoint_epoch_3_macroF1_0.5734_falseF1_0.1884.pt


/tmp/ipykernel_31/118544494.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Loaded checkpoint from epoch 3
Stored validation loss: 0.0734
Computed validation loss: 0.0734
Stored metrics: {'True_precision': 0.9677661169415293, 'False_precision': 0.15853658536585366, 'True_recall': 0.9492647058823529, 'False_recall': 0.23214285714285715, 'True_fscore': 0.9584261321455085, 'False_fscore': 0.1884057971014493, 'micro_precision': 0.9209039548022598, 'micro_recall': 0.9209039548022598, 'micro_fscore': 0.9209039548022597, 'macro_precision': 0.5631513511536914, 'macro_recall': 0.590703781512605, 'macro_fscore': 0.5734159646234789, 'confusion_matrix': array([[  13,   43],
       [  69, 1291]])}
Computed metrics: {'True_precision': 0.9677661169415293, 'False_precision': 0.15853658536585366, 'True_recall': 0.9492647058823529, 'False_recall': 0.23214285714285715, 'True_fscore': 0.9584261321455085, 'False_fscore': 0.1884057971014493, 'micro_precision': 0.9209039548022598, 'micro_recall': 0.9209039548022598, 'micro_fscore': 0.9209039548022597, 'macro_precision': 0.5631513511

/tmp/ipykernel_31/118544494.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Loaded checkpoint from epoch 13
Stored validation loss: 0.0945
Computed validation loss: 0.0729
Stored metrics: {'True_precision': 0.9634873323397913, 'False_precision': 0.0945945945945946, 'True_recall': 0.950735294117647, 'False_recall': 0.125, 'True_fscore': 0.9570688378978534, 'False_fscore': 0.1076923076923077, 'micro_precision': 0.9180790960451978, 'micro_recall': 0.9180790960451978, 'micro_fscore': 0.9180790960451978, 'macro_precision': 0.529040963467193, 'macro_recall': 0.5378676470588235, 'macro_fscore': 0.5323805727950806, 'confusion_matrix': array([[   7,   49],
       [  67, 1293]])}
Computed metrics: {'True_precision': 0.9634873323397913, 'False_precision': 0.0945945945945946, 'True_recall': 0.950735294117647, 'False_recall': 0.125, 'True_fscore': 0.9570688378978534, 'False_fscore': 0.1076923076923077, 'micro_precision': 0.9180790960451978, 'micro_recall': 0.9180790960451978, 'micro_fscore': 0.9180790960451978, 'macro_precision': 0.529040963467193, 'macro_recall': 0.537867

...

/tmp/ipykernel_31/118544494.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Loaded checkpoint from epoch 4
Stored validation loss: 0.0761
Computed validation loss: 0.0761
Stored metrics: {'True_precision': 0.9656972408650261, 'False_precision': 0.13333333333333333, 'True_recall': 0.9522058823529411, 'False_recall': 0.17857142857142858, 'True_fscore': 0.9589041095890409, 'False_fscore': 0.15267175572519082, 'micro_precision': 0.9216101694915254, 'micro_recall': 0.9216101694915254, 'micro_fscore': 0.9216101694915254, 'macro_precision': 0.5495152870991797, 'macro_recall': 0.5653886554621849, 'macro_fscore': 0.5557879326571159, 'confusion_matrix': array([[  10,   46],
       [  65, 1295]])}
Computed metrics: {'True_precision': 0.9656972408650261, 'False_precision': 0.13333333333333333, 'True_recall': 0.9522058823529411, 'False_recall': 0.17857142857142858, 'True_fscore': 0.9589041095890409, 'False_fscore': 0.15267175572519082, 'micro_precision': 0.9216101694915254, 'micro_recall': 0.9216101694915254, 'micro_fscore': 0.9216101694915254, 'macro_precision': 0.5495152

/tmp/ipykernel_31/118544494.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Loaded checkpoint from epoch 2
Stored validation loss: 0.0999
Computed validation loss: 0.0759
Stored metrics: {'True_precision': 0.9605654761904762, 'False_precision': 0.041666666666666664, 'True_recall': 0.9492647058823529, 'False_recall': 0.05357142857142857, 'True_fscore': 0.9548816568047337, 'False_fscore': 0.046875, 'micro_precision': 0.9138418079096046, 'micro_recall': 0.9138418079096046, 'micro_fscore': 0.9138418079096046, 'macro_precision': 0.5011160714285714, 'macro_recall': 0.5014180672268908, 'macro_fscore': 0.5008783284023668, 'confusion_matrix': array([[   3,   53],
       [  69, 1291]])}
Computed metrics: {'True_precision': 0.9617604617604618, 'False_precision': 0.1, 'True_recall': 0.9801470588235294, 'False_recall': 0.05357142857142857, 'True_fscore': 0.9708667152221414, 'False_fscore': 0.06976744186046512, 'micro_precision': 0.943502824858757, 'micro_recall': 0.943502824858757, 'micro_fscore': 0.943502824858757, 'macro_precision': 0.530880230880231, 'macro_recall': 0.5

AssertionError: Mismatch in True_precision

In [53]:


# File paths
train_file = "/kaggle/input/dataset-deception/train.jsonl"
val_file = "/kaggle/input/dataset-deception/validation.jsonl"
test_file = "/kaggle/input/dataset-deception/test.jsonl"
checkpoint_dir = 'checkpoints'

# Preprocess train data first to get maps and stats
use_ling_features = True
train_data, country_map, season_map, year_map, ling_stats = preprocess_data(train_file, use_ling_features=use_ling_features)

# Preprocess val and test with train's maps and stats
val_data, _, _, _, _ = preprocess_data(val_file, country_map, season_map, year_map, use_ling_features=use_ling_features, ling_stats=ling_stats)
test_data, _, _, _, _ = preprocess_data(test_file, country_map, season_map, year_map, use_ling_features=use_ling_features, ling_stats=ling_stats)

# Initialize model parameters
num_countries = len(country_map)
num_seasons = len(season_map)
num_years = len(year_map)

# Load the best checkpoint
checkpoint_path = os.path.join(checkpoint_dir, 'best_checkpoint.pt')
if not os.path.exists(checkpoint_path):
    # If no best checkpoint, find the one with highest macro F1
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pt')]
    if not checkpoint_files:
        raise FileNotFoundError(f"No checkpoint files found in {checkpoint_dir}")
    
    # Extract metrics from checkpoint names (assuming they follow the naming convention)
    best_macro_f1 = 0
    best_checkpoint = None
    for cf in checkpoint_files:
        match = re.search(r'macroF1_([0-9\.]+)', cf)
        if match:
            macro_f1 = float(match.group(1))
            if macro_f1 > best_macro_f1:
                best_macro_f1 = macro_f1
                best_checkpoint = cf
    
    if best_checkpoint:
        checkpoint_path = os.path.join(checkpoint_dir, best_checkpoint)
        logger.info(f"Using checkpoint with highest macro F1: {best_checkpoint}")
    else:
        # Just use the first checkpoint if naming convention doesn't match
        checkpoint_path = os.path.join(checkpoint_dir, checkpoint_files[0])
        logger.info(f"Using first available checkpoint: {checkpoint_files[0]}")

logger.info(f"Loading checkpoint from {checkpoint_path}")
checkpoint = torch.load(checkpoint_path, map_location=device)

# Initialize the model with the same parameters
model = EnhancedDeceptionDetector(
    num_countries=num_countries, num_seasons=num_seasons, num_years=num_years,
    sentence_emb_dim=384, ling_feature_dim=6 if use_ling_features else 0,
    country_emb_dim=16, season_emb_dim=8, year_emb_dim=8, hidden_dim=64, dropout=0.4
).to(device)

# Load model weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Define evaluation function
def evaluate_checkpoint(model, data, batch_size=8):
    total_loss = 0
    all_preds, all_labels = [], []
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([checkpoint.get('pos_weight', 1.0)], device=device), reduction='none')
    
    with torch.no_grad():
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            messages = [c['messages'] for c in batch]
            ling_features = [c['ling_features'] for c in batch]
            senders = torch.nn.utils.rnn.pad_sequence([c['speakers'] for c in batch], batch_first=True, padding_value=model.country_embedding.padding_idx).to(device)
            receivers = torch.nn.utils.rnn.pad_sequence([c['receivers'] for c in batch], batch_first=True, padding_value=model.country_embedding.padding_idx).to(device)
            seasons = torch.nn.utils.rnn.pad_sequence([c['seasons'] for c in batch], batch_first=True, padding_value=model.season_embedding.padding_idx).to(device)
            years = torch.nn.utils.rnn.pad_sequence([c['years'] for c in batch], batch_first=True, padding_value=model.year_embedding.padding_idx).to(device)
            game_scores = torch.nn.utils.rnn.pad_sequence([c['game_scores'] for c in batch], batch_first=True, padding_value=0).to(device)
            game_score_deltas = torch.nn.utils.rnn.pad_sequence([c['game_score_deltas'] for c in batch], batch_first=True, padding_value=0).to(device)
            sender_labels = torch.nn.utils.rnn.pad_sequence([c['sender_labels'] for c in batch], batch_first=True, padding_value=-1).to(device)
            lengths = [len(c['messages']) for c in batch]

            final_pred = model(messages, ling_features, senders, receivers, seasons, years,
                              game_scores, game_score_deltas, lengths)
            
            loss = criterion(final_pred.squeeze(-1), sender_labels)
            mask = (sender_labels != -1)
            loss = (loss * mask).sum() / (mask.sum() + 1e-8)
            total_loss += loss.item()

            all_preds.append(final_pred.squeeze(-1)[mask])
            all_labels.append(sender_labels[mask])

    avg_loss = total_loss / (len(data) / batch_size)
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    metrics = compute_metrics(all_preds, all_labels)
    return avg_loss, metrics

# Evaluate on all datasets
logger.info("Evaluating model on training data...")
train_loss, train_metrics = evaluate_checkpoint(model, train_data)

logger.info("Evaluating model on validation data...")
val_loss, val_metrics = evaluate_checkpoint(model, val_data)

logger.info("Evaluating model on test data...")
test_loss, test_metrics = evaluate_checkpoint(model, test_data)

# Prepare results dictionary similar to training output
results = {
    'test_metrics': test_metrics,
    'training_loss': train_loss,
    'validation_loss': val_loss,
    'test_loss': test_loss,
    'best_validation_loss': checkpoint.get('val_loss', val_loss),
    'best_validation_metrics': checkpoint.get('metrics', val_metrics)
}

# Print checkpoint info
print(f"\n📋 Checkpoint Information")
print(f"Checkpoint file: {os.path.basename(checkpoint_path)}")
if 'epoch' in checkpoint:
    print(f"Epoch: {checkpoint['epoch']}")

# Print results
print("\n=== Per-Class Metrics (for 'True' and 'False' classes) ===")
print("Precision")
print(f"True_precision: {results['test_metrics']['True_precision']:.4f}")
print(f"False_precision: {results['test_metrics']['False_precision']:.4f}")
print("Recall")
print(f"True_recall: {results['test_metrics']['True_recall']:.4f}")
print(f"False_recall: {results['test_metrics']['False_recall']:.4f}")
print("F1-Score")
print(f"True_fscore: {results['test_metrics']['True_fscore']:.4f}")
print(f"False_fscore: {results['test_metrics']['False_fscore']:.4f}")

print("\n📊 Micro-Averaged Metrics")
print(f"micro_precision: {results['test_metrics']['micro_precision']:.4f}")
print(f"micro_recall: {results['test_metrics']['micro_recall']:.4f}")
print(f"micro_fscore: {results['test_metrics']['micro_fscore']:.4f}")

print("\n📈 Macro-Averaged Metrics")
print(f"macro_precision: {results['test_metrics']['macro_precision']:.4f}")
print(f"macro_recall: {results['test_metrics']['macro_recall']:.4f}")
print(f"macro_fscore: {results['test_metrics']['macro_fscore']:.4f}")

print("\n🧠 Loss")
print(f"training_loss: {results['training_loss']:.4f}")
print(f"validation_loss: {results['validation_loss']:.4f}")
if 'val_loss' in checkpoint:
    print(f"best_validation_loss: {results['best_validation_loss']:.4f}")
print(f"test_loss: {results['test_loss']:.4f}")

# Print saved validation metrics if available
if 'metrics' in checkpoint:
    print("\n🧪 Best Validation Metrics (from checkpoint)")
    for metric in ['True_precision', 'False_precision', 'True_recall', 'False_recall', 
                  'True_fscore', 'False_fscore', 'micro_precision', 'micro_recall', 
                  'micro_fscore', 'macro_precision', 'macro_recall', 'macro_fscore']:
        if metric in checkpoint['metrics']:
            print(f"best_validation_{metric}: {checkpoint['metrics'][metric]:.4f}")

print("\n🔍 Test Confusion Matrix")
print(results['test_metrics']['confusion_matrix'])

# Evaluate all checkpoints if there are multiple
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pt') and f != 'best_checkpoint.pt']
if len(checkpoint_files) > 1:
    print("\n📊 Comparing All Checkpoints (Test Macro F1)")
    checkpoint_results = []
    
    for cf in checkpoint_files:
        cp_path = os.path.join(checkpoint_dir, cf)
        cp = torch.load(cp_path, map_location=device)
        model.load_state_dict(cp['model_state_dict'])
        model.eval()
        
        _, cp_test_metrics = evaluate_checkpoint(model, test_data)
        checkpoint_results.append({
            'name': cf,
            'macro_f1': cp_test_metrics['macro_fscore'],
            'false_f1': cp_test_metrics['False_fscore']
        })
    
    # Sort by macro F1 score
    checkpoint_results.sort(key=lambda x: x['macro_f1'], reverse=True)
    for idx, res in enumerate(checkpoint_results):
        print(f"{idx+1}. {res['name']}: Macro F1 = {res['macro_f1']:.4f}, False F1 = {res['false_f1']:.4f}")


Processing conversations:   1%|          | 2/189 [00:02<03:50,  1.23s/it]


KeyboardInterrupt: 